# Método das Duas Fases — Simplex Parametrizado

Este notebook complementa a teoria apresentada na Célula 1, estruturando o método das duas fases com foco na montagem do dicionário artificial, nos quadros do simplex e na resolução automatizada (em estilo Solver do Excel) dos exercícios propostos.

## Como usar este notebook
- Preencha o dicionário `exercicios` com os coeficientes de cada problema.
- Execute `resolver_duas_fases` para gerar dicionários artificiais, quadros simplex e a solução ótima (quando existir).
- Utilize as funções de exibição para acompanhar os pivôs e montar relatórios dos dicionários artificial inicial, modificado e finais de cada fase.

In [10]:
FORCAR_ARTIFICIAIS_EM_MENOR_IGUAL = True

In [11]:
from dataclasses import dataclass, field
from fractions import Fraction
from typing import Any, Dict, List, Optional, Sequence, Tuple

try:
    import pandas as pd  # type: ignore
except ImportError:  # pragma: no cover
    pd = None

try:
    from IPython.display import Markdown, display  # type: ignore
except ImportError:  # pragma: no cover
    Markdown = None
    display = None


def to_fraction(value: Any) -> Fraction:
    """Converte o valor informado para Fraction."""
    if isinstance(value, Fraction):
        return value
    if isinstance(value, int):
        return Fraction(value, 1)
    if isinstance(value, float):
        return Fraction(str(value))
    if isinstance(value, str):
        texto = value.strip()
        if not texto:
            raise ValueError("Valor textual vazio não pode ser convertido em fração.")
        return Fraction(texto)
    raise TypeError(f"Tipo não suportado para conversão em Fraction: {type(value)}")


def fraction_to_string(value: Fraction) -> str:
    """Formata uma fração como string legível."""
    value = value.limit_denominator()
    if value.denominator == 1:
        return str(value.numerator)
    return f"{value.numerator}/{value.denominator}"


def cleanup_coeffs(coeffs: Dict[str, Fraction]) -> Dict[str, Fraction]:
    """Remove coeficientes nulos preservando a ordem de inserção."""
    return {var: coef for var, coef in coeffs.items() if coef != 0}


def ordered_unique_append(seq: List[str], item: str) -> None:
    """Adiciona item mantendo a ordem e evitando duplicatas."""
    if item not in seq:
        seq.append(item)


In [12]:
@dataclass
class ConstraintData:
    coefficients: Dict[str, Fraction]
    sense: str
    rhs: Fraction

    def __post_init__(self) -> None:
        self.coefficients = {k: to_fraction(v) for k, v in self.coefficients.items()}
        self.sense = self.sense.strip()
        if self.sense not in ("<=", ">=", "="):
            raise ValueError(f"Sentido de restrição inválido: {self.sense}")
        self.rhs = to_fraction(self.rhs)


@dataclass
class ProblemData:
    name: str
    sense: str
    objective: Dict[str, Fraction]
    constraints: List[ConstraintData]

    def __post_init__(self) -> None:
        self.objective = {k: to_fraction(v) for k, v in self.objective.items()}
        self.sense = self.sense.lower().strip()
        if self.sense not in ("max", "min"):
            raise ValueError(f"Tipo de objetivo inválido: {self.sense}")


def make_constraint(coeffs: Dict[str, Any], sense: str, rhs: Any) -> ConstraintData:
    """Atalho para declarar restrições com sintaxe enxuta."""
    return ConstraintData(coefficients=coeffs, sense=sense, rhs=rhs)


def make_problem(name: str, sense: str, objective: Dict[str, Any], constraints: Sequence[ConstraintData]) -> ProblemData:
    """Atalho para declarar problemas lineares."""
    return ProblemData(name=name, sense=sense, objective=objective, constraints=list(constraints))


In [13]:
@dataclass
class DictionaryRow:
    constant: Fraction
    coeffs: Dict[str, Fraction] = field(default_factory=dict)

    def clone(self) -> "DictionaryRow":
        return DictionaryRow(self.constant, dict(self.coeffs))

    def cleanup(self) -> None:
        self.coeffs = cleanup_coeffs(self.coeffs)

    def scaled(self, factor: Fraction) -> "DictionaryRow":
        return DictionaryRow(
            constant=self.constant * factor,
            coeffs={var: coef * factor for var, coef in self.coeffs.items()},
        )


class SimplexDictionary:
    def __init__(
        self,
        rows: Dict[str, DictionaryRow],
        nonbasic_vars: Sequence[str],
        var_order: Sequence[str],
        objective: DictionaryRow,
        name: str = "Z",
    ) -> None:
        self.rows = {var: row.clone() for var, row in rows.items()}
        self.var_order = list(dict.fromkeys(var_order))
        self.objective = objective.clone()
        self.name = name
        self._refresh_sets(nonbasic_vars)

    def _refresh_sets(self, nonbasic_vars: Optional[Sequence[str]] = None) -> None:
        self.basic = set(self.rows.keys())
        if nonbasic_vars is None:
            candidatos = [var for var in self.var_order if var not in self.basic]
        else:
            candidatos = list(nonbasic_vars)
        self.nonbasic = set(var for var in candidatos if var not in self.basic)
        for var in self.var_order:
            if var not in self.basic and var not in self.nonbasic:
                self.nonbasic.add(var)

    def clone(self, name: Optional[str] = None) -> "SimplexDictionary":
        return SimplexDictionary(
            rows={var: row.clone() for var, row in self.rows.items()},
            nonbasic_vars=list(self.nonbasic),
            var_order=list(self.var_order),
            objective=self.objective.clone(),
            name=name or self.name,
        )

    def basis_order(self) -> List[str]:
        return [var for var in self.var_order if var in self.basic]

    def nonbasic_order(self) -> List[str]:
        return [var for var in self.var_order if var in self.nonbasic]

    def choose_entering(self, maximize: bool = True) -> Optional[str]:
        candidatos: List[str] = []
        for var in self.var_order:
            if var not in self.nonbasic:
                continue
            coeff = self.objective.coeffs.get(var, Fraction(0))
            if maximize and coeff > 0:
                candidatos.append(var)
            elif not maximize and coeff < 0:
                candidatos.append(var)
        if not candidatos:
            return None
        for var in self.var_order:
            if var in candidatos:
                return var
        return candidatos[0]

    def choose_leaving(self, entering: str) -> Optional[str]:
        leaving: Optional[str] = None
        best_ratio: Optional[Fraction] = None
        for var in self.var_order:
            if var not in self.basic:
                continue
            row = self.rows[var]
            coeff = row.coeffs.get(entering, Fraction(0))
            if coeff >= 0:
                continue
            ratio = row.constant / (-coeff)
            if ratio < 0:
                continue
            if best_ratio is None or ratio < best_ratio:
                best_ratio = ratio
                leaving = var
            elif ratio == best_ratio and leaving is not None:
                if self.var_order.index(var) < self.var_order.index(leaving):
                    leaving = var
        return leaving

    def pivot(self, leaving: str, entering: str) -> None:
        if leaving not in self.rows:
            raise KeyError(f"Variável básica {leaving} não encontrada.")
        row = self.rows[leaving]
        alpha = row.coeffs.get(entering, Fraction(0))
        if alpha == 0:
            raise ZeroDivisionError(f"Coeficiente nulo para a coluna de entrada {entering}.")
        inv_alpha = Fraction(1, 1) / alpha

        new_const = -row.constant * inv_alpha
        new_coeffs: Dict[str, Fraction] = {}
        for var, coef in row.coeffs.items():
            if var == entering:
                continue
            new_coeffs[var] = -coef * inv_alpha
        new_coeffs[leaving] = inv_alpha
        new_row = DictionaryRow(new_const, cleanup_coeffs(new_coeffs))

        self.rows.pop(leaving)
        self.rows[entering] = new_row

        for basic_var, other_row in self.rows.items():
            if basic_var == entering:
                continue
            beta = other_row.coeffs.get(entering, Fraction(0))
            if beta == 0:
                continue
            other_row.constant += beta * new_row.constant
            for var_name, gamma in new_row.coeffs.items():
                other_row.coeffs[var_name] = other_row.coeffs.get(var_name, Fraction(0)) + beta * gamma
            other_row.coeffs.pop(entering, None)
            other_row.cleanup()

        beta = self.objective.coeffs.get(entering, Fraction(0))
        if beta != 0:
            self.objective.constant += beta * new_row.constant
            for var_name, gamma in new_row.coeffs.items():
                self.objective.coeffs[var_name] = self.objective.coeffs.get(var_name, Fraction(0)) + beta * gamma
            self.objective.coeffs.pop(entering, None)
            self.objective.cleanup()

        if leaving not in self.var_order:
            self.var_order.append(leaving)
        self._refresh_sets()

    def objective_value(self) -> Fraction:
        return self.objective.constant


In [14]:
def normalize_constraint(constraint: ConstraintData) -> ConstraintData:
    coeffs = dict(constraint.coefficients)
    sense = constraint.sense
    rhs = constraint.rhs
    if rhs < 0:
        coeffs = {var: -coef for var, coef in coeffs.items()}
        rhs = -rhs
        if sense == "<=":
            sense = ">="
        elif sense == ">=":
            sense = "<="
    return ConstraintData(coefficients=coeffs, sense=sense, rhs=rhs)


def build_initial_dictionary(
    problem: ProblemData,
    force_leq_artificial: bool = FORCAR_ARTIFICIAIS_EM_MENOR_IGUAL,
) -> Tuple[SimplexDictionary, Dict[str, Any]]:
    normalized_constraints = [normalize_constraint(constraint) for constraint in problem.constraints]
    var_order: List[str] = []
    original_vars: List[str] = []

    for var in problem.objective.keys():
        ordered_unique_append(var_order, var)
        ordered_unique_append(original_vars, var)

    rows: Dict[str, DictionaryRow] = {}
    slack_vars: List[str] = []
    surplus_vars: List[str] = []
    artificial_vars: List[str] = []
    var_kinds: Dict[str, str] = {var: "original" for var in original_vars}
    raw_rows: List[Dict[str, Any]] = []

    slack_count = 0
    surplus_count = 0
    artificial_count = 0

    for constraint in normalized_constraints:
        row_coeffs: Dict[str, Fraction] = {var: Fraction(0) for var in var_order}
        for var, coef in constraint.coefficients.items():
            ordered_unique_append(var_order, var)
            row_coeffs[var] = coef

        rhs = constraint.rhs
        sense = constraint.sense
        basis_var: Optional[str] = None

        if sense == "<=":
            slack_count += 1
            slack_name = "F" if slack_count == 1 else f"F{slack_count}"
            ordered_unique_append(var_order, slack_name)
            row_coeffs[slack_name] = Fraction(1)
            slack_vars.append(slack_name)
            var_kinds[slack_name] = "slack"
            if force_leq_artificial:
                artificial_count += 1
                art_name = f"A{artificial_count}"
                ordered_unique_append(var_order, art_name)
                row_coeffs[art_name] = Fraction(1)
                basis_var = art_name
                artificial_vars.append(art_name)
                var_kinds[art_name] = "artificial"
            else:
                basis_var = slack_name
        elif sense == ">=":
            surplus_count += 1
            surplus_name = "E" if surplus_count == 1 else f"E{surplus_count}"
            ordered_unique_append(var_order, surplus_name)
            row_coeffs[surplus_name] = Fraction(-1)
            surplus_vars.append(surplus_name)
            var_kinds[surplus_name] = "surplus"

            artificial_count += 1
            art_name = f"A{artificial_count}"
            ordered_unique_append(var_order, art_name)
            row_coeffs[art_name] = Fraction(1)
            basis_var = art_name
            artificial_vars.append(art_name)
            var_kinds[art_name] = "artificial"
        else:  # igualdade
            artificial_count += 1
            art_name = f"A{artificial_count}"
            ordered_unique_append(var_order, art_name)
            row_coeffs[art_name] = Fraction(1)
            basis_var = art_name
            artificial_vars.append(art_name)
            var_kinds[art_name] = "artificial"

        if basis_var is None:
            raise RuntimeError("Nenhuma variável básica identificada para a restrição.")

        eq_coeffs = {var: coef for var, coef in row_coeffs.items() if coef != 0}
        eq_coeffs[basis_var] = Fraction(1)
        raw_rows.append({
            "base": basis_var,
            "coeffs": cleanup_coeffs(eq_coeffs),
            "rhs": rhs,
        })

        coeffs = {}
        for var, coef in row_coeffs.items():
            if var == basis_var:
                continue
            if coef != 0:
                coeffs[var] = -coef
        rows[basis_var] = DictionaryRow(rhs, cleanup_coeffs(coeffs))

    nonbasic_vars = [var for var in var_order if var not in rows]
    dictionary = SimplexDictionary(
        rows=rows,
        nonbasic_vars=nonbasic_vars,
        var_order=var_order,
        objective=DictionaryRow(Fraction(0), {}),
        name="Base inicial",
    )

    metadata: Dict[str, Any] = {
        "var_order": list(var_order),
        "original_vars": original_vars,
        "slack_vars": slack_vars,
        "surplus_vars": surplus_vars,
        "artificial_vars": artificial_vars,
        "var_kinds": var_kinds,
        "normalized_constraints": normalized_constraints,
        "problem_name": problem.name,
        "objective_sense": problem.sense,
        "problem": problem,
        "raw_rows": raw_rows,
        "force_leq_artificial": force_leq_artificial,
    }
    return dictionary, metadata


def compute_artificial_objectives(
    dictionary: SimplexDictionary, artificial_vars: Sequence[str]
) -> Tuple[DictionaryRow, DictionaryRow, DictionaryRow]:
    w_initial = DictionaryRow(
        constant=Fraction(0),
        coeffs={var: Fraction(1) for var in artificial_vars},
    )
    w_const = Fraction(0)
    w_coeffs: Dict[str, Fraction] = {}
    for art in artificial_vars:
        if art in dictionary.rows:
            row = dictionary.rows[art]
            w_const += row.constant
            for var, coef in row.coeffs.items():
                w_coeffs[var] = w_coeffs.get(var, Fraction(0)) + coef
        else:
            w_coeffs[art] = w_coeffs.get(art, Fraction(0)) + Fraction(1)
    w_modified = DictionaryRow(w_const, cleanup_coeffs(w_coeffs))
    w_prime = DictionaryRow(
        -w_modified.constant,
        {var: -coef for var, coef in w_modified.coeffs.items()},
    )
    w_prime.cleanup()
    return w_initial, w_modified, w_prime


def remove_artificials(dictionary: SimplexDictionary, artificial_vars: Sequence[str]) -> Dict[str, Any]:
    relatorio = {"removidas": [], "pivos_realizados": []}
    for art in list(artificial_vars):
        if art in dictionary.basic:
            row = dictionary.rows[art]
            pivotado = False
            for candidato in dictionary.var_order:
                if candidato == art or candidato in artificial_vars:
                    continue
                coeff = row.coeffs.get(candidato, Fraction(0))
                if coeff != 0:
                    dictionary.pivot(art, candidato)
                    relatorio["pivos_realizados"].append({"artificial": art, "entrando": candidato})
                    pivotado = True
                    break
            if not pivotado:
                if row.constant == 0 and all(coef == 0 for coef in row.coeffs.values()):
                    dictionary.rows.pop(art)
                else:
                    raise ValueError("Variável artificial permaneceu básica com valor positivo após a fase 1.")
        dictionary.objective.coeffs.pop(art, None)
        dictionary.nonbasic.discard(art)
        if art in dictionary.rows:
            dictionary.rows.pop(art)
        if art in dictionary.var_order:
            dictionary.var_order.remove(art)
        relatorio["removidas"].append(art)
        dictionary._refresh_sets()
    return relatorio


def build_phase2_objective(dictionary: SimplexDictionary, problem: ProblemData) -> Tuple[DictionaryRow, Fraction]:
    multiplier = Fraction(1) if problem.sense == "max" else Fraction(-1)
    coeffs = {var: Fraction(0) for var in dictionary.var_order}
    for var, coef in problem.objective.items():
        coeffs[var] = to_fraction(coef) * multiplier
    objective = DictionaryRow(Fraction(0), coeffs)
    for basic_var, row in dictionary.rows.items():
        coef = objective.coeffs.get(basic_var, Fraction(0))
        if coef != 0:
            objective.constant += coef * row.constant
            for var, value in row.coeffs.items():
                objective.coeffs[var] = objective.coeffs.get(var, Fraction(0)) + coef * value
            objective.coeffs.pop(basic_var, None)
    objective.cleanup()
    for basic_var in list(objective.coeffs.keys()):
        if basic_var in dictionary.basic:
            objective.coeffs.pop(basic_var, None)
    objective.cleanup()
    return objective, multiplier


def run_simplex(
    dictionary: SimplexDictionary,
    maximize: bool = True,
    max_iter: int = 50,
) -> Tuple[str, List[Dict[str, Any]]]:
    historico: List[Dict[str, Any]] = []
    historico.append({"iteracao": 0, "dicionario": dictionary.clone(name=dictionary.name)})
    for passo in range(1, max_iter + 1):
        entrando = dictionary.choose_entering(maximize=maximize)
        if entrando is None:
            return "otimo", historico
        saindo = dictionary.choose_leaving(entrando)
        if saindo is None:
            historico.append({
                "iteracao": passo,
                "dicionario": dictionary.clone(name=dictionary.name),
                "pivot": {"entrando": entrando, "saindo": None},
            })
            return "ilimitado", historico
        dictionary.pivot(saindo, entrando)
        historico.append({
            "iteracao": passo,
            "dicionario": dictionary.clone(name=dictionary.name),
            "pivot": {"entrando": entrando, "saindo": saindo},
        })
    return "max_iter", historico


def resolver_duas_fases(
    problem: ProblemData,
    max_iter: int = 50,
    force_leq_artificial: bool = FORCAR_ARTIFICIAIS_EM_MENOR_IGUAL,
) -> Dict[str, Any]:
    base_dict, meta = build_initial_dictionary(problem, force_leq_artificial=force_leq_artificial)
    meta["force_leq_artificial"] = force_leq_artificial
    fase1_base = base_dict.clone(name="Base inicial")
    w_inicial, w_modificado, w_prime = compute_artificial_objectives(fase1_base, meta["artificial_vars"])

    if meta["artificial_vars"]:
        fase1_dict = fase1_base.clone(name="Fase 1 (inicial)")
        fase1_dict.objective = w_prime.clone()
        fase1_dict.name = "W'"
        status1, historico1 = run_simplex(fase1_dict, maximize=True, max_iter=max_iter)
        w_final = -fase1_dict.objective.constant
        fase1 = {
            "status": status1,
            "historico": historico1,
            "w_inicial": w_inicial,
            "w_modificado": w_modificado,
            "w_prime": w_prime,
            "w_valor_final": w_final,
            "dicionario_base": fase1_base.clone(name="Base inicial"),
        }
        if status1 == "max_iter":
            return {
                "status": "interrompido",
                "fase1": fase1,
                "fase2": None,
                "motivo": "Número máximo de iterações atingido na fase 1.",
                "metadados": meta,
                "problema": problem,
            }
        if status1 == "ilimitado":
            return {
                "status": "falha",
                "fase1": fase1,
                "fase2": None,
                "motivo": "Função artificial ilimitada. Revise a formulação do problema.",
                "metadados": meta,
                "problema": problem,
            }
        if w_final > Fraction(0):
            return {
                "status": "inviavel",
                "fase1": fase1,
                "fase2": None,
                "motivo": "Soma das artificiais diferente de zero ao final da fase 1.",
                "metadados": meta,
                "problema": problem,
            }
        fase2_dict = historico1[-1]["dicionario"].clone(name="Pré Fase 2")
    else:
        fase1 = {
            "status": "nao_aplicavel",
            "historico": [{"iteracao": 0, "dicionario": fase1_base.clone(name="Base inicial")}],
            "w_inicial": w_inicial,
            "w_modificado": w_modificado,
            "w_prime": w_prime,
            "w_valor_final": Fraction(0),
            "dicionario_base": fase1_base.clone(name="Base inicial"),
        }
        fase2_dict = fase1_base.clone(name="Pré Fase 2")

    remocao_info = remove_artificials(fase2_dict, meta["artificial_vars"])
    fase2_objetivo, multiplier = build_phase2_objective(fase2_dict, problem)
    fase2_dict.objective = fase2_objetivo
    fase2_dict.name = "Z"

    status2, historico2 = run_simplex(fase2_dict, maximize=True, max_iter=max_iter)
    fase2 = {
        "status": status2,
        "historico": historico2,
        "remocao_artificiais": remocao_info,
    }

    if status2 == "max_iter":
        return {
            "status": "interrompido",
            "fase1": fase1,
            "fase2": fase2,
            "motivo": "Número máximo de iterações atingido na fase 2.",
            "metadados": meta,
            "problema": problem,
        }
    if status2 == "ilimitado":
        return {
            "status": "ilimitado",
            "fase1": fase1,
            "fase2": fase2,
            "motivo": "Função objetivo ilimitada na fase 2.",
            "metadados": meta,
            "problema": problem,
        }

    final_dict = historico2[-1]["dicionario"].clone(name="Dicionário final")
    objetivo_transformado = final_dict.objective.constant
    objetivo_original = objetivo_transformado if problem.sense == "max" else -objetivo_transformado

    valores_vars: Dict[str, Fraction] = {}
    for var in meta["original_vars"]:
        if var in final_dict.rows:
            valores_vars[var] = final_dict.rows[var].constant
        else:
            valores_vars[var] = Fraction(0)

    resultado = {
        "status": "otimo",
        "fase1": fase1,
        "fase2": fase2,
        "solucao": {
            "variaveis": valores_vars,
            "variaveis_decimais": {var: float(valor) for var, valor in valores_vars.items()},
            "objetivo": objetivo_original,
        },
        "metadados": meta,
        "problema": problem,
        "dicionario_final": final_dict,
        "multiplicador_objetivo": multiplier,
    }
    return resultado


In [39]:
def quadro_para_dataframe(
    dictionary: SimplexDictionary,
    objetivo: Optional[DictionaryRow] = None,
    rotulo_objetivo: Optional[str] = None,
    colunas: Optional[Sequence[str]] = None,
):
    alvo = objetivo if objetivo is not None else dictionary.objective
    if colunas is None:
        colunas_calculadas: List[str] = []
        for var in dictionary.var_order:
            if var in dictionary.nonbasic or alvo.coeffs.get(var, Fraction(0)) != 0:
                colunas_calculadas.append(var)
        colunas = colunas_calculadas
    else:
        colunas = list(colunas)

    cabecalho = ["Variável básica", "Termo livre"] + list(colunas)
    linhas: List[List[str]] = []
    for var in dictionary.basis_order():
        row = dictionary.rows[var]
        linha = [var, fraction_to_string(row.constant)]
        for col in colunas:
            linha.append(fraction_to_string(row.coeffs.get(col, Fraction(0))))
        linhas.append(linha)

    rotulo = rotulo_objetivo or dictionary.name
    linha_obj = [rotulo, fraction_to_string(alvo.constant)]
    for col in colunas:
        linha_obj.append(fraction_to_string(alvo.coeffs.get(col, Fraction(0))))
    linhas.append(linha_obj)

    if pd is not None:
        return pd.DataFrame(linhas, columns=cabecalho)
    return cabecalho, linhas


def exibir_quadro(
    dictionary: SimplexDictionary,
    objetivo: Optional[DictionaryRow] = None,
    rotulo_objetivo: Optional[str] = None,
    titulo: Optional[str] = None,
    colunas: Optional[Sequence[str]] = None,
) -> None:
    tabela = quadro_para_dataframe(dictionary, objetivo=objetivo, rotulo_objetivo=rotulo_objetivo, colunas=colunas)
    if pd is not None and isinstance(tabela, pd.DataFrame):
        if titulo:
            if display and Markdown:
                display(Markdown(f"#### {titulo}"))
            else:
                print(f"\n#### {titulo}")
        if display:
            display(tabela)
        else:
            print(tabela.to_string(index=False))
    else:
        cabecalho, linhas = tabela
        if titulo:
            print(f"\n#### {titulo}")
        print("\t".join(cabecalho))
        for linha in linhas:
            print("\t".join(linha))


def mostrar_dicionarios_artificiais(resultado: Dict[str, Any]) -> None:
    meta = resultado.get("metadados", {})
    raw_rows = meta.get("raw_rows", [])
    var_order = meta.get("var_order", [])
    artificial_vars = set(meta.get("artificial_vars", []))

    if raw_rows:
        colunas = list(var_order)
        cabecalho = ["Base"] + colunas + ["Res"]
        linhas: List[List[str]] = []
        for row in raw_rows:
            linha = [row["base"]]
            for var in colunas:
                linha.append(fraction_to_string(row["coeffs"].get(var, Fraction(0))))
            linha.append(fraction_to_string(row["rhs"]))
            linhas.append(linha)

        problema: Optional[ProblemData] = meta.get("problem") or resultado.get("problema")
        if problema is not None:
            linha_z = ["Z"]
            for var in colunas:
                linha_z.append(fraction_to_string(to_fraction(problema.objective.get(var, 0))))
            linha_z.append("0")
            linhas.append(linha_z)

        if artificial_vars:
            w_rhs = Fraction(0)
            w_coeffs: Dict[str, Fraction] = {var: Fraction(0) for var in colunas}
            for row in raw_rows:
                if row["base"] not in artificial_vars:
                    continue
                w_rhs += row["rhs"]
                for var, coef in row["coeffs"].items():
                    if var == row["base"]:
                        continue
                    w_coeffs[var] = w_coeffs.get(var, Fraction(0)) + coef
            if any(coef != 0 for coef in w_coeffs.values()) or w_rhs != 0:
                linha_w = ["W"]
                for var in colunas:
                    linha_w.append(fraction_to_string(w_coeffs.get(var, Fraction(0))))
                linha_w.append(fraction_to_string(w_rhs))
                linhas.append(linha_w)

                linha_w_neg = ["-W"]
                for var in colunas:
                    linha_w_neg.append(fraction_to_string(-w_coeffs.get(var, Fraction(0))))
                linha_w_neg.append(fraction_to_string(-w_rhs))
                linhas.append(linha_w_neg)

        if pd is not None:
            df_raw = pd.DataFrame(linhas, columns=cabecalho)
            if display and Markdown:
                display(Markdown("#### Dicionário Artificial Inicial (forma original)"))
                display(df_raw)
            else:
                print("#### Dicionário Artificial Inicial (forma original)")
                print(df_raw.to_string(index=False))
        else:
            print("#### Dicionário Artificial Inicial (forma original)")
            print("\t".join(cabecalho))
            for linha in linhas:
                print("\t".join(linha))
    else:
        print("Dicionário artificial inicial indisponível (problema sem artificiais).")

    fase1 = resultado.get("fase1", {})
    historico = fase1.get("historico", [])
    if not historico:
        print("Fase 1 não executada para este problema.")
        return

    dicionario_inicial: SimplexDictionary = historico[0]["dicionario"].clone()
    w_modificado: DictionaryRow = fase1.get("w_modificado", DictionaryRow(Fraction(0), {}))
    w_prime: DictionaryRow = fase1.get("w_prime", DictionaryRow(Fraction(0), {}))

    colunas_canon: List[str] = []
    for var in dicionario_inicial.var_order:
        if var in dicionario_inicial.nonbasic or w_modificado.coeffs.get(var, Fraction(0)) != 0 or w_prime.coeffs.get(var, Fraction(0)) != 0:
            colunas_canon.append(var)

    exibir_quadro(
        dicionario_inicial,
        objetivo=w_modificado,
        rotulo_objetivo="W",
        titulo="Dicionário Artificial Modificado",
        colunas=colunas_canon,
    )
    exibir_quadro(
        dicionario_inicial,
        objetivo=w_prime,
        rotulo_objetivo="W",
        titulo="Dicionário W' (utilizado na Fase 1)",
        colunas=colunas_canon,
    )


def mostrar_iteracoes(resultado: Dict[str, Any], fase: str = "fase1") -> None:
    dados_fase = resultado.get(fase, {})
    historico = dados_fase.get("historico", [])
    if not historico:
        print(f"Nenhum histórico disponível para {fase}.")
        return
    for item in historico:
        titulo = f"{fase.upper()} — iteração {item['iteracao']}"
        exibir_quadro(item["dicionario"], titulo=titulo)


def avaliar_exercicio(
    codigo: str,
    mostrar: bool = True,
    max_iter: int = 50,
    force_leq_artificial: Optional[bool] = None,
) -> Dict[str, Any]:
    if codigo not in exercicios:
        raise KeyError(f"Exercício {codigo} não cadastrado.")
    problema = exercicios[codigo]
    resultado = resolver_duas_fases(
        problema,
        max_iter=max_iter,
        force_leq_artificial=force_leq_artificial if force_leq_artificial is not None else FORCAR_ARTIFICIAIS_EM_MENOR_IGUAL,
    )
    if mostrar:
        print(f"Exercício {codigo} — status: {resultado['status']}")
        if resultado["status"] == "otimo":
            objetivo = fraction_to_string(resultado["solucao"]["objetivo"])
            valores = {var: fraction_to_string(valor) for var, valor in resultado["solucao"]["variaveis"].items()}
            print(f"Valor ótimo: {objetivo}")
            print(f"Solução (frações): {valores}")
        elif resultado["status"] == "inviavel":
            print("Problema inviável: a fase 1 terminou com soma de artificiais positiva.")
        elif resultado["status"] == "interrompido":
            print(resultado.get("motivo"))
    return resultado


In [40]:
# Exercícios prontos para uso — ajuste conforme necessário
exercicios: Dict[str, ProblemData] = {
    "1a": make_problem(
        name="1a",
        sense="max",
        objective={"x": 4, "y": 3},
        constraints=[
            make_constraint({"x": 2, "y": 4}, "<=", 15),
            make_constraint({"x": 1, "y": 1}, "=", 10),
        ],
    ),
    "1b": make_problem(
        name="1a",
        sense="min",
        objective={"x": 4, "y": 3, "z": 9},
        constraints=[
            make_constraint({"x": 2, "y": 4, "z": 6}, "<=", 15),
            make_constraint({"x": 1, "y": 1, "z": 1}, "=", 9/2),
            make_constraint({"x": 6, "y": 1, "z": 6}, ">=", 12),
        ],
    )
}


In [41]:
resultado = avaliar_exercicio("1b")
mostrar_dicionarios_artificiais(resultado)
# mostrar_iteracoes(resultado_1a, fase="fase1")
# mostrar_iteracoes(resultado_1a, fase="fase2")


Exercício 1b — status: otimo
Valor ótimo: 15
Solução (frações): {'x': '3/2', 'y': '3', 'z': '0'}


#### Dicionário Artificial Inicial (forma original)

,Base,x,y,z,F,A1,A2,E,A3,Res
0,A1,2,4,6,1,1,0,0,0,15
1,A2,1,1,1,0,0,1,0,0,9/2
2,A3,6,1,6,0,0,0,-1,1,12
3,Z,4,3,9,0,0,0,0,0,0
4,W,9,6,13,1,0,0,-1,0,63/2
5,-W,-9,-6,-13,-1,0,0,1,0,-63/2


#### Dicionário Artificial Modificado

,Variável básica,Termo livre,x,y,z,F,E
0,A1,15,-2,-4,-6,-1,0
1,A2,9/2,-1,-1,-1,0,0
2,A3,12,-6,-1,-6,0,1
3,W,63/2,-9,-6,-13,-1,1


#### Dicionário W' (utilizado na Fase 1)

,Variável básica,Termo livre,x,y,z,F,E
0,A1,15,-2,-4,-6,-1,0
1,A2,9/2,-1,-1,-1,0,0
2,A3,12,-6,-1,-6,0,1
3,W,-63/2,9,6,13,1,-1


### Próximos passos
- Ajuste ou acrescente problemas em `exercicios` conforme necessário.
- Utilize `avaliar_exercicio(codigo)` para gerar os dicionários e a solução.
- Exporte resultados com `exibir_quadro` ou converta os `DataFrame` retornados em planilhas para documentação.
